In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm.notebook as tqdm
from multiprocessing import Pool
import seaborn as sns
import subprocess
import os
import pickle
import json

import py.data_gen as data_gen
import py.algos as algos

os.makedirs('figures', exist_ok=True)

# IMPORTANT: please set MODE, N_CORES (if MODE=='compute') and exp_dir_load (if MODE=='load')
MODE = 'compute'
assert MODE in ('compute', 'load')
N_CORES = 1 # if MODE=='compute', how many cores to use.
exp_dir_load = "experiments/3848a72" # if MODE=='load', then set the directory to load results from.

plt.rcParams.update({"font.size": 18})
%load_ext autoreload
%autoreload 2

In [ ]:
# some helper functions
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def save_runs(exp_root, exp_name, runs, args):
    git_hash = algos.get_git_revision_hash()
    dirty = b"script.py" in subprocess.check_output(["git", "diff", "--name-only"])
    exp_root = os.path.join(exp_root, f"{git_hash}-dirty" if dirty else f"{git_hash}")
    exp_dir = os.path.join(exp_root, exp_name)
    os.makedirs(exp_dir)
    with open(os.path.join(exp_dir, "results"), "wb") as f:
        pickle.dump(runs, f)
    with open(os.path.join(exp_dir, "args"), "w") as f:
        json.dump(vars(args), f, indent=True, sort_keys=True, cls=NumpyEncoder)


def get_accepted_per_round(_runs, n_rounds):
    _list1 = []
    for _r in _runs:
        accepted_per_round = np.array(np.array_split(_r, n_rounds)).sum(axis=-1)
        rejected_per_round = 1 - accepted_per_round
        _list1.append(accepted_per_round)
    accepted_per_round = np.concatenate(_list1)
    return accepted_per_round

## harris et. al setup with selection

In [ ]:
# plot data.
cmd = f"--num-applicants 100000 --applicants-per-round 1000 --clip --envs-accept-rates .25 --rank-type uniform"
args = algos.get_args(cmd)
pref_vect = args.pref_vect
np.random.seed(1)
(
    b,
    x,
    y,
    EW,
    theta,
    w,
    z,
    y_hat,
    adv_idx,
    disadv_idx,
    g,
    theta_star,
) = data_gen.generate_data(
    args.num_applicants, args.applicants_per_round, args.fixed_effort_conversion, args
)

cmd = f"--num-applicants 100000 --applicants-per-round 1000 --clip --envs-accept-rates .25 --rank-type prediction"
args = algos.get_args(cmd)
pref_vect = args.pref_vect
np.random.seed(1)
(
    b,
    x,
    y2,
    EW,
    theta,
    w,
    z2,
    y_hat,
    adv_idx,
    disadv_idx,
    g,
    theta_star,
) = data_gen.generate_data(
    args.num_applicants, args.applicants_per_round, args.fixed_effort_conversion, args
)
fig, ax = plt.subplots()
plt.hist(y2[0], alpha=0.2, color="k", bins=50)
plt.hist(y2[0][z2 == 1], color="g", bins=50, alpha=0.3, label="sampling by rank")
plt.hist(y[0][z == 1], color="b", bins=50, alpha=0.3, label="sampling uniformly")
ax.grid()
ax.set_xlabel("College GPA ($Y_t$)")
ax.set_ylabel("Count")
ax.legend()

# assert np.all (y2[0] ==y[0] )
plt.tight_layout()
plt.savefig("figures/haris-et-al-selection-dist.pdf")

# experiment: 2SLS under selection bias. 

In [ ]:
if MODE=='compute':
    n_runs = 10 
    # ranking selection
    cmd = f"--num-applicants 100000 --applicants-per-round 1000 --clip --methods ols 2sls --envs-accept-rate .25 --rank-type prediction"
    args = algos.get_args(cmd)
    args_list = [(s, args) for s in np.arange(n_runs)]
    with Pool(N_CORES) as p:
        runs = p.starmap(algos.run_multi_env, args_list)
    runs = [r[0] for r in runs]

    # uniform sampling
    cmd = f"--num-applicants 100000 --applicants-per-round 1000 --clip --methods ols 2sls --envs-accept-rate .25 --rank-type uniform"
    args = algos.get_args(cmd)
    args_list = [(s, args) for s in np.arange(n_runs)]
    with Pool(N_CORES) as p:
        runs2 = p.starmap(algos.run_multi_env, args_list)
    runs2 = [r[0] for r in runs2]

    # saving stuff.
    try:
        save_runs("experiments", "harris-with-selection", (runs, runs2), args)
    except FileExistsError:
        pass
else:
    # loading saved stuff
    with open(os.path.join(exp_dir_load, "harris-with-selection", "results"), "rb") as f:
        runs = pickle.load(f)
        runs, runs2 = runs

In [ ]:
df = algos.runs2df(runs)
df["sampling"] = "sampling by rank"
df2 = algos.runs2df(runs2)
df2["sampling"] = "sampling uniformly"
df = pd.concat((df, df2))
# converting to long format.
# df = script.runs2df(runs)
dflong = pd.melt(
    df,
    id_vars=("iterations", "sampling"),
    value_vars=("ols_env0", "2sls_env0"),
    var_name="method",
    value_name="error",
)
dflong["method"] = dflong["method"].astype("category")
dflong["method"] = dflong["method"].cat.rename_categories(
    {"ols_env0": "OLS", "2sls_env0": "2SLS"}
)

dflong["iterations"] = dflong["iterations"] + 2
# plot
fig, ax = plt.subplots()
sns.lineplot(
    data=dflong, x="iterations", y="error", hue="method", style="sampling", ax=ax
)
ax.set_ylim(bottom=-0.001, top=0.2)
ax.legend(loc="upper right")
ax.set_ylabel(r"$||\theta^* - \hat{\theta}^*|| $")
ax.set_xlabel("Rounds (t)")
ax.grid()
ax.set_xlim((0, 100))
handles, labels = ax.get_legend_handles_labels()
handles = [handles[1], handles[2], handles[4], handles[5]]
labels = [labels[1], labels[2], labels[4], labels[5]]
ax.legend(
    handles=handles,
    labels=labels,
    ncol=2,
    loc="lower center",
    bbox_to_anchor=(0.25, 1, 0.5, 0.5),
)

plt.tight_layout()
plt.savefig("figures/harris-et-al-selection-results.pdf")

# our setup: data generation

In [ ]:
cmd = f"--num-applicants 100000 --applicants-per-round 1000 --scaled-duplicates sequence --fixed-effort-conversion --methods 2sls ols ours --envs-accept-rate .5 --clip --alpha 1"

args = algos.get_args(cmd)
pref_vect = args.pref_vect
np.random.seed(1)
(
    b,
    x,
    y,
    EW,
    theta,
    w,
    z,
    y_hat,
    adv_idx,
    disadv_idx,
    g,
    theta_star,
) = data_gen.generate_data(
    args.num_applicants, args.applicants_per_round, args.fixed_effort_conversion, args
)

In [ ]:
# x.shape, b.shape, y.shape 

# idx = 1
# # max_cov = np.max(np.concat rx[:, idx], 
# max_cov = np.max (np.concatenate ((x[:, idx], b[:, idx])))
# min_cov = np.min (np.concatenate ((x[:, idx], b[:, idx])))

# min_cov, max_cov 

In [ ]:
# dataframe with disadvantaged students.
df0 = pd.DataFrame(
    {
        r"$b_t^{SAT}$": b[disadv_idx, 0],
        r"$b_t^{HS GPA}$": b[disadv_idx, 1],
        r"$x_t^{SAT}$": x[disadv_idx, 0],
        r"$x_t^{HS GPA}$": x[disadv_idx, 1],
        "status": z[disadv_idx],
        "y": y[0, disadv_idx],
        "y_hat": y_hat[0, disadv_idx],
    }
)

# dataframe with advantaged students.
df1 = pd.DataFrame(
    {
        r"$b_t^{SAT}$": b[adv_idx, 0],
        r"$b_t^{HS GPA}$": b[adv_idx, 1],
        r"$x_t^{SAT}$": x[adv_idx, 0],
        r"$x_t^{HS GPA}$": x[adv_idx, 1],
        "status": z[adv_idx],
        "y": y[0, adv_idx],
        "y_hat": y_hat[0, adv_idx],
    }
)

df0["class"] = "disadv"
df1["class"] = "adv"
df = pd.concat((df0, df1))
df["status"] = df.status.astype("category").cat.rename_categories(
    {0.0: "rejected", 1.0: "accepted"}
)
df["status"]

# SAT score
dflong = pd.melt(
    df,
    id_vars=("class"),
    value_vars=(r"$b_t^{SAT}$", r"$x_t^{SAT}$"),
    var_name="time",
    value_name="score",
)
dflong

fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True)
g = sns.histplot(dflong[dflong["class"] == "disadv"], x="score", hue="time", ax=ax[0])
g.legend_.set_title(None)
g = sns.histplot(dflong[dflong["class"] == "adv"], x="score", hue="time", ax=ax[1])
g.legend_.set_title(None)
ax[1].set_xlabel("SAT score")


# four mean lines
ax[0].axvline(
    x=dflong[
        (dflong["class"] == "disadv") & (dflong.time == r"$b_t^{SAT}$")
    ].score.mean(),
    color=sns.color_palette()[0],
    linestyle="--",
)
ax[0].axvline(
    x=dflong[
        (dflong["class"] == "disadv") & (dflong.time == r"$x_t^{SAT}$")
    ].score.mean(),
    color=sns.color_palette()[1],
    linestyle="--",
)

ax[1].axvline(
    x=dflong[(dflong["class"] == "adv") & (dflong.time == r"$b_t^{SAT}$")].score.mean(),
    color=sns.color_palette()[0],
    linestyle="--",
)
ax[1].axvline(
    x=dflong[(dflong["class"] == "adv") & (dflong.time == r"$x_t^{SAT}$")].score.mean(),
    color=sns.color_palette()[1],
    linestyle="--",
)

plt.tight_layout()
plt.savefig("figures/our-settings-data1.png")

# GPA
dflong = pd.melt(
    df,
    id_vars=("class"),
    value_vars=(r"$b_t^{HS GPA}$", r"$x_t^{HS GPA}$"),
    var_name="time",
    value_name="score",
)
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True)
g = sns.histplot(dflong[dflong["class"] == "disadv"], x="score", hue="time", ax=ax[0])
g.legend_.set_title(None)
g = sns.histplot(dflong[dflong["class"] == "adv"], x="score", hue="time", ax=ax[1])
g.legend_.set_title(None)
ax[1].set_xlabel("High School GPA")

# four mean lines
ax[0].axvline(
    x=dflong[
        (dflong["class"] == "disadv") & (dflong.time == r"$b_t^{HS GPA}$")
    ].score.mean(),
    color=sns.color_palette()[0],
    linestyle="--",
)
ax[0].axvline(
    x=dflong[
        (dflong["class"] == "disadv") & (dflong.time == r"$x_t^{HS GPA}$")
    ].score.mean(),
    color=sns.color_palette()[1],
    linestyle="--",
)

ax[1].axvline(
    x=dflong[
        (dflong["class"] == "adv") & (dflong.time == r"$b_t^{HS GPA}$")
    ].score.mean(),
    color=sns.color_palette()[0],
    linestyle="--",
)
ax[1].axvline(
    x=dflong[
        (dflong["class"] == "adv") & (dflong.time == r"$x_t^{HS GPA}$")
    ].score.mean(),
    color=sns.color_palette()[1],
    linestyle="--",
)

plt.tight_layout()
plt.savefig("figures/our-settings-data2.png")

# dfa, dfr = pd.DataFrame({'y': y[0][z==1]}), pd.DataFrame({'y': y[0][z==0]})
# df = pd.concat((dfa, dfr))
fig, ax = plt.subplots()
g = sns.histplot(df, x="y", hue="status", ax=ax, multiple="stack")
g.legend_.set_title(None)
ax.set_xlabel("College GPA")
ax.grid()

plt.tight_layout()
plt.savefig("figures/our-settings-data3.pdf")

# experiment: single environment.

In [ ]:
if MODE=='compute':
    n_runs = 10 
    args = algos.get_args(cmd)
    args_list = [(s, args) for s in np.arange(n_runs)]
    with Pool(N_CORES) as p:
        runs = p.starmap(algos.run_multi_env, args_list)
        # _runs = [r[2] for r in runs]
    runs = [r[0] for r in runs]

    # saving stuff.
    try:
        save_runs("experiments", "our-settings-single-env", runs, args)
    except FileExistsError:
        pass
else:
    # loading saved stuff
    with open(os.path.join(exp_dir_load, "our-settings-single-env", "results"), "rb") as f:
        runs = pickle.load(f)
    runs
    df2 = algos.runs2df(runs)
    df = df2

In [ ]:
df = algos.runs2df(runs)
# long format for plotting
value_vars = [f"{m}_env{e}" for m in args.methods for e in range(args.num_envs)]
dflong = pd.melt(
    df, id_vars="iterations", value_vars=value_vars, var_name="env", value_name="error"
)
dflong
dflong["method"] = dflong.env.apply(lambda x: "_".join(x.split("_")[:-1]))
dflong["env"] = dflong.env.apply(lambda x: x.split("_")[-1])
dflong["method"] = dflong["method"].astype("category")
dflong["method"] = dflong["method"].cat.rename_categories(
    {"ours": "MSLR", "ols": "OLS", "2sls": "2SLS"}
)
dflong
dflong_stack = dflong  # remember for later.
dflong_stack["iterations"] = dflong_stack["iterations"] + 2
dflong_stack
plt.rcParams.update({"font.size": 18})
fig, ax = plt.subplots(figsize=(7, 3.5))
sns.lineplot(
    dflong_stack,
    x="iterations",
    y="error",
    errorbar=("ci", 95),
    ax=ax,
    hue="method",
    markers={"MSLR": "^", "OLS": "X", "2SLS": "o"},
    markevery=10,
    markersize=9,
)
ax.grid()
ax.set_ylim(bottom=-0.001, top=0.3)
ax.set_ylabel(r"$|| \theta^* - \hat{\theta}^* || $")
ax.set_xlabel("Rounds (t)")

handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles[:],
    labels=labels[:],
    ncol=3,
    loc="lower left",
    bbox_to_anchor=(0, 1, 0.5, 0.5),
)
ax.set_xlim((0, 100))
plt.tight_layout()
plt.savefig("figures/our-settings-single-env.pdf")

# experiment: sensitivity analysis (linearity assumption)

In [ ]:
if MODE=='compute':
    dfs = []
    for alpha in (0, .25, .5, .75, 1):
        n_runs = 10 
        cmd = f"--num-applicants 100000 --applicants-per-round 1000 --scaled-duplicates sequence --fixed-effort-conversion --methods 2sls ols ours --envs-accept-rate .5 --clip --alpha {alpha}"
        args = algos.get_args(cmd)
        args_list = [(s, args) for s in np.arange(n_runs)]
        with Pool(N_CORES) as p:
            runs = p.starmap(algos.run_multi_env, args_list)
            # _runs = [r[2] for r in runs]
        runs = [r[0] for r in runs]

        df = algos.runs2df(runs)
        df['alpha'] = alpha
        dfs.append(df)

    df = pd.concat(dfs)
    df = df[df['iterations']==df.iterations.max()] # filtering out the result of final iteration

    # saving stuff.
    try:
        save_runs("experiments", "linearity-sens-single-env-causal", dfs, args)
    except FileExistsError:
        pass
elif MODE=='load':
    with open('experiments/76b75cf/linearity-sens-single-env-causal/results', 'rb') as f:
        dfs = pickle.load(f)
    df = pd.concat(dfs)
    df = df[df['iterations']==df.iterations.max()]

In [ ]:
# long format, for plotting.
value_vars = [f"{m}_env{e}" for m in args.methods for e in range(args.num_envs)]
dflong = pd.melt(
    df, id_vars=("alpha",), value_vars=value_vars, var_name="env", value_name="error"
)
dflong
dflong["method"] = dflong.env.apply(lambda x: "_".join(x.split("_")[:-1]))
dflong["env"] = dflong.env.apply(lambda x: x.split("_")[-1])

dflong["method"] = dflong["method"].astype("category")
dflong["method"] = dflong["method"].cat.rename_categories(
    {"ours": "MSLR", "ols": "OLS", "2sls": "2SLS"}
)
dflong 

sns.pointplot(
    data=dflong, x='alpha', y='error', hue='method'
)

# experiment: robustness on acceptance rate. 

In [ ]:
if MODE=='compute':
    ## selection function variation.
    n_envs = 1
    dfs = []
    df_w = []
    df_z = []
    for env0_accept_rate in (0.2, 0.4, 0.6, 0.8, 1.0):
        cmd = f"--num-applicants 500000 --applicants-per-round 1000 --scaled-duplicates sequence --fixed-effort-conversion --normalize --methods 2sls ols ours --offline-eval --envs-accept-rates {env0_accept_rate}"
        args = algos.get_args(cmd)

        n_runs = 10
        args_list = [(s, args) for s in np.arange(n_runs)]
        with Pool(N_CORES) as p:
            runs_data = p.starmap(algos.run_multi_env, args_list)

        # recording errors
        runs = [r[0] for r in runs_data]
        df = algos.runs2df(runs)
        df["env0_accept_rate"] = env0_accept_rate
        dfs.append(df)

        # recording z
        runs_z = [r[2] for r in runs_data]
        accepted_per_round = get_accepted_per_round(
            runs_z, args.num_applicants / args.applicants_per_round
        )
        df = pd.DataFrame(
            {"accepted_per_round": accepted_per_round, "env0_accept_rate": env0_accept_rate}
        )
        df["rejected_per_round"] = args.applicants_per_round - df["accepted_per_round"]
        df_z.append(df)

    df = pd.concat(dfs)
    # df_w = pd.concat(df_w)
    df_z = pd.concat(df_z)
    value_vars = [f"{m}_env{ei}" for m in args.methods for ei in range(args.num_envs)]
    value_vars
    dflong = pd.melt(
        df,
        id_vars="env0_accept_rate",
        value_vars=value_vars,
        value_name="error",
        var_name="method_env",
    )
    dflong["method"] = dflong.method_env.apply(lambda x: "_".join(x.split("_")[:-1]))
    dflong["env"] = dflong.method_env.apply(lambda x: x.split("_")[-1])

    try:
        save_runs("experiments", "our-settings-single-env-ablation", dflong, args)
    except FileExistsError:
        pass
else: # load
    with open(
        os.path.join(exp_dir_load, "our-settings-single-env-ablation", "results"), "rb"
    ) as f:
        dflong = pickle.load(f)

In [ ]:
fig, ax = plt.subplots()
dflong["method"] = dflong["method"].astype("category")
dflong["method"] = dflong["method"].cat.rename_categories(
    {"2sls": "2SLS", "ols": "OLS", "ours": "MSLR"}
)
sns.pointplot(
    data=dflong[dflong.env == "env0"],
    x="env0_accept_rate",
    y="error",
    hue="method",
    markers="^",
)
ax.grid()

handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles[:],
    labels=labels[:],
    ncol=3,
    loc="lower center",
    bbox_to_anchor=(0.5, 1),
)

ax.set_xlabel(r"$\rho$")
ax.set_ylabel(r"$|| \theta^* - \hat{\theta}^*|| $")
plt.tight_layout()
ax.invert_xaxis()
plt.savefig("figures/our-setttings-single-env-ablation.pdf")

## our setup for multiple envs: data generation.

In [ ]:
cmd = f"--num-applicants 100000 --applicants-per-round 1000 --scaled-duplicates sequence --fixed-effort-conversion --normalize --methods ours --num-envs 2 --theta-star-std 0.1 --envs-accept-rate .5 .5 --num-cooperative-envs 2"
args = algos.get_args(cmd)
pref_vect = args.pref_vect 
np.random.seed(3)
(
    b,
    x,
    y,
    EW,
    theta,
    w,
    z,
    y_hat,
    adv_idx,
    disadv_idx,
    g,
    theta_star,
) = data_gen.generate_data(
    args.num_applicants, args.applicants_per_round, args.fixed_effort_conversion, args
)

In [ ]:
def add_envs(d, idx, var, prefix):
    d.update(
        {
            f"{prefix}_env{env_idx}": var[env_idx, idx]
            for env_idx in range(args.num_envs)
        }
    )
    return d


# disadvantage class
df0 = {
    "b1": b[disadv_idx, 0],
    "b2": b[disadv_idx, 1],
    "status": z[disadv_idx],
    "y": y[0, disadv_idx],
    "y_hat": y_hat[0, disadv_idx],
    "x1": x[disadv_idx, 0],
    "x2": x[disadv_idx, 1],
}
df0 = add_envs(df0, disadv_idx, y, "y")
df0 = add_envs(df0, disadv_idx, w, "w")
df0.keys()

df0 = pd.DataFrame(df0)
df0["class"] = "disadv"

# advantaged class
df1 = {
    "b1": b[adv_idx, 0],
    "b2": b[adv_idx, 1],
    "status": z[adv_idx],
    "y": y[0, adv_idx],
    "y_hat": y_hat[0, adv_idx],
    "x1": x[adv_idx, 0],
    "x2": x[adv_idx, 1],
}
df1 = add_envs(df1, adv_idx, y, "y")
df1 = add_envs(df1, adv_idx, w, "w")
df1.keys()

df1 = pd.DataFrame(df1)
df1["class"] = "adv"
df = pd.concat((df0, df1))
df["status"] = df["status"].astype("category")
df
fig, ax = plt.subplots()

# first plot
df_z = pd.get_dummies(df["status"]).to_numpy()
df_z = np.array(
    np.split(df_z, (args.num_applicants / args.applicants_per_round), axis=0)
)
ax.axhline(y=args.applicants_per_round, color="k", linestyle="--")

assert df_z.shape == (
    (args.num_applicants / args.applicants_per_round),
    args.applicants_per_round,
    args.num_envs + 1,
), f"{df_z.shape}"
df_z = df_z.sum(axis=1)
sns.barplot(pd.DataFrame(df_z), ax=ax)
ax.set_xlabel(r"$z_t$")

# second plot
fig, ax = plt.subplots()
df = df.rename(columns={"status": r"$Z_t$"})
# df[r'$Z_t$'] = df[r'$Z_t$'].cat.rename_categories({0.0: '0', 1.0: '1', 2.0: '2'})

sns.histplot(df[df[r"$Z_t$"] != 0], x="x1", hue=r"$Z_t$", ax=ax)
ax.axvline(x=df[df[r"$Z_t$"] == 0].x1.mean(), color="b", linestyle="--")
ax.axvline(x=df[df[r"$Z_t$"] == 1].x1.mean(), color="y", linestyle="-.")
ax.axvline(x=df[df[r"$Z_t$"] == 2].x1.mean(), color="g", linestyle="--")
ax.axvline(x=df[df[r"$Z_t$"] == 3].x1.mean(), color="r", linestyle="--")
ax.set_xlabel(r"$x_t^{(1)}$ (SAT score)")

plt.tight_layout()
plt.savefig("figures/our-settings-multi1.png")

# third plot
fig, ax = plt.subplots()
sns.histplot(df[df[r"$Z_t$"] != 0], x="x2", hue=r"$Z_t$", ax=ax)
for env_idx, c in zip(range(args.num_envs + 1), ("b", "y", "g", "r")):
    print(df[df[r"$Z_t$"] == env_idx].x2.mean())
    ax.axvline(x=df[df[r"$Z_t$"] == env_idx].x2.mean(), c=c, linestyle="--")

ax.set_xlabel(r"$x_t^{(2)}$ (HS GPA)")
plt.tight_layout()
plt.savefig("figures/our-settings-multi2.png")

# experiment: protocol vs. no protocol.

In [ ]:
# some helper functions. 
def plot_convergence(
    df, y_col, y_range, y_label, hline_y, hline_label=None, pathname=None
):
    _, ax = plt.subplots(figsize=(7, 3.5))
    sns.lineplot(
        df[df.env == 1],
        x="iterations",
        y=y_col,
        hue="is-protocol",
        linestyle="dashdot",
        legend=False,
    )
    sns.lineplot(df[df.env == 0], x="iterations", y=y_col, hue="is-protocol")
    ax.set_ylim(y_range)
    ax.axhline(y=hline_y, color="k", linestyle="--", label=hline_label)

    ax.set_ylabel(y_label)
    ax.set_xlabel(r"Rounds (t)")
    ax.grid()
    handles, labels = ax.get_legend_handles_labels()
    lgd = ax.legend(
        handles=handles[:],
        labels=labels[:],
        ncol=3,
        loc="lower center",
        bbox_to_anchor=(0.5, 1),
        handlelength=1,
        labelspacing=0,
    )
    plt.tight_layout()
    ax.set_xlim(0, 300)

    if pathname is not None:
        plt.savefig(pathname)


def get_data(runs, env_idx):
    """Gets a list of tuple. Each tupel is as returned by the function algos.run_multi_env.

    Args:
        runs (List):  A list of n_runs elements. Each element is a tuple as
                               returned by the function algos.run_multi_env.
        env_idx (integer): Index of envrionment for which to return the data.

    Returns:
        df: A dataframe with columns ['iterations', 'est_dim*', 'err_dim*', 'env']
            having estimate for causal parameter per iteration.
        Number of rows are (n_runs x n_iterations of the algorithm.)
    """
    dfs = []
    for r in runs:  # iterator
        ests = np.array(r[1][f"ours_env{env_idx}"])
        theta_star = r[-1]["theta_star"]
        err = ests - theta_star[env_idx, :]
        assert err.ndim == 2
        df = pd.DataFrame(
            {
                "env": env_idx,
                "err_dim0": err[:, 0],
                "err_dim1": err[:, 1],
                "est_dim0": ests[:, 0],
                "est_dim1": ests[:, 1],
            }
        )
        dfs.append(df)
    df = pd.concat(dfs)
    df.reset_index(inplace=True)
    df.rename({"index": "iterations"}, axis=1, inplace=True)
    return df



In [ ]:
if MODE=='compute':
    theta_star_std = 0.1
    _runs = []
    for nce in tqdm.tqdm((2, 1)):
        cmd = f"--num-applicants 300000 --applicants-per-round 1000 --scaled-duplicates sequence --fixed-effort-conversion --normalize --methods ours --num-envs 2 --theta-star-std {theta_star_std} --envs-accept-rates .5 --num-cooperative-envs {nce}"

        args = algos.get_args(cmd)
        n_runs = 50
        args_list = [(s, args) for s in np.arange(n_runs)]
        with Pool(N_CORES) as p:
            runs = p.starmap(algos.run_multi_env, args_list)
        _runs.append(runs)

    runs_protocol = _runs[0]
    runs_no_protocol = _runs[1]

    # some post processing before saving, to reduce the filesize.
    runs_protocol = _runs[0]
    runs_no_protocol = _runs[1]

    dfs = []
    for env_idx in (0, 1):
        df = get_data(runs_protocol, env_idx)
        df2 = get_data(runs_no_protocol, env_idx)
        df["is-protocol"] = "Co-op"
        df2["is-protocol"] = "No co-op."
        df = pd.concat((df, df2))
        dfs.append(df)

    df = pd.concat(dfs)
    df.reset_index(inplace=True)
    df
    try:
        save_runs("experiments", "p-vs-np", df, args)
    except FileExistsError:
        pass
else:
    with open(os.path.join(exp_dir_load, "p-vs-np", "results"), "rb") as f:
        df = pickle.load(f)

In [ ]:
plot_convergence(
    df,
    "err_dim1",
    (-0.05, 0.05),
    r"$\hat{\theta}^{HS GPA}_{i} - \theta^{*, HS GPA}_{i}$",
    0,
    pathname="figures/protocol-vs-no-protocol4.pdf",
)
plot_convergence(
    df=df,
    y_col="err_dim0",
    y_range=(-0.001, 0.001),
    y_label=r"$\hat{\theta}^{SAT}_{i} - \theta^{*, SAT}_{i}$",
    hline_y=0,
    pathname="figures/protocol-vs-no-protocol3.pdf",
)

# experiment: protocol improvement 

In [ ]:
if MODE=='compute':
    theta_star_std = 0.1  # set to 0 for the same underlying causal parameters.
    _runs = []
    n_envs = 3
    coop_envs = (3, 2)
    for nce in tqdm.tqdm(coop_envs):
        cmd = (
            f"--offline-eval --num-applicants 500000 --applicants-per-round 5000 --scaled-duplicates sequence "
            f"--fixed-effort-conversion --normalize --methods ours --num-envs {n_envs} --theta-star-std {theta_star_std} "
            f"--envs-accept-rates .5 --num-cooperative-envs {nce}"
        )
        args = algos.get_args(cmd)
        n_runs = 100
        args_list = [(s, args) for s in np.arange(n_runs)]
        with Pool(N_CORES) as p:
            runs = p.starmap(algos.run_multi_env, args_list)
            _runs.append(runs)
    # some post-processing to save some disk space upon saving.
    dfs = []
    for i, nce in enumerate((coop_envs)):
        df = algos.runs2df([r[0] for r in _runs[i]])
        df["cooperative envs."] = (
            r"Full Cooperation" if nce == 3 else r"Partial Cooperation"
        )
        dfs.append(df)
    df = pd.concat(dfs)
    df["ours_env0+ours_env1"] = (df["ours_env0"] + df["ours_env1"]) / 2.0
    df

    try:
        save_runs("experiments", "protocol-improvement", df, args)
    except FileExistsError:
        pass
else:
    with open(os.path.join(exp_dir_load, "protocol-improvement", "results"), "rb") as f:
        df = pickle.load(f)

In [ ]:
value_vars = (f"ours_env{i}" for i in range(3))
dflong = pd.melt(
    df,
    id_vars=("iterations", "cooperative envs."),
    value_vars=value_vars,
    var_name="env",
    value_name="error",
)
dflong

rename_cats_args = {f"ours_env{env_idx}": f"${env_idx+1}$" for env_idx in range(3)}
dflong["env"] = dflong.env.astype("category").cat.rename_categories(rename_cats_args)
dflong
fig, ax = plt.subplots(figsize=(7, 3.5))
sns.barplot(
    dflong,
    x="env",
    hue="cooperative envs.",
    y="error",
    palette={
        "Full Cooperation": sns.color_palette()[1],
        "Partial Cooperation": sns.color_palette()[0],
    },
    hue_order=("Partial Cooperation", "Full Cooperation"),
)
# ax.legend(bbox_to_anchor=(1,1), title='cooperative envs.')
ax.grid()
ax.set_ylabel(r"$|| \theta^* - \hat{\theta}^*||$")
ax.set_xlabel("Decision Maker")

handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles[:],
    labels=labels[:],
    ncol=2,
    loc="lower center",
    bbox_to_anchor=(0.5, 1),
    handlelength=1,
    labelspacing=0,
)
ax.set_xticklabels(("A", "B", "C"))

plt.tight_layout()
plt.savefig("figures/protocol-vs-no-protocol-group.pdf")

# experiment: utility in one env.

In [ ]:
if MODE=='compute':
    cmd = (
        f"--num-applicants 100000 --applicants-per-round 1000 --scaled-duplicates sequence "
        "--fixed-effort-conversion --normalize --methods ols ours --envs-accept-rate .5 --theta-star-std 0"
    )
    cmd = f"{cmd} --offline-eval"

    args = algos.get_args(cmd)
    combos = [("theta_ao_hat",), ("theta_star_hat",), ("theta_ols_hat",)]

    dfs = []
    for combo in combos:
        n_runs = 100
        args_list = [(args, t, combo) for t in range(n_runs)]
        with Pool(N_CORES) as p:
            results = p.starmap(algos.run_multi_env_utility, args_list)

        results = [r[0] for r in results]  # filtering out the test theta array.
        results = np.array(results)
        pd_dict = {
            f"perf_env{env_idx}": results[:, env_idx] for env_idx in range(args.num_envs)
        }
        pd_dict.update(
            {f"test_theta_env{env_idx}": combo[env_idx] for env_idx in range(args.num_envs)}
        )
        df = pd.DataFrame(data=pd_dict)
        dfs.append(df)
    df = pd.concat(dfs)

    try:
        save_runs("experiments", "utility-single-env", df, args)
    except FileExistsError:
        pass
else:
    with open(os.path.join(exp_dir_load, "utility-single-env", "results"), "rb") as f:
        df = pickle.load(f)

In [ ]:
# fig, ax = plt.subplots()
# sns.pointplot(df, y="perf_env0", x="test_theta_env0", join=False)
# ax.set_ylabel(r"$\mathcal{Q}(\theta)$")
# ax.set_xticklabels(
    # labels=[r"$\hat{\theta}_{AO}$", r"$\hat{\theta}^*$", r"$\hat{\theta}_{OLS}$"]
# )
# ax.grid()

df_test = df.groupby("test_theta_env0").agg(("mean", "sem"))
df_test

# experiment: utility over multiple envs.

In [ ]:
if MODE=='compute':
    theta_star_std = 0.1
    from itertools import product

    options = ("theta_ao_hat", "theta_ols_hat", "theta_star_hat")
    combos = [x for x in product(options, options)]

    cmd = (
        f"--num-applicants 500000 --applicants-per-round 1000 --scaled-duplicates sequence "
        f"--fixed-effort-conversion --normalize --methods ours ols --num-envs 2 --theta-star-std {theta_star_std} "
        f"--envs-accept-rates .5 --num-cooperative-envs 2"
    )
    cmd = f"{cmd} --offline-eval"
    args = algos.get_args(cmd)
    dfs = []
    for combo in tqdm.tqdm(combos):
        n_runs = 100
        args_list = [(args, t, combo) for t in range(n_runs)]
        with Pool(N_CORES) as p:
            results = p.starmap(algos.run_multi_env_utility, args_list)

        results = [r[0] for r in results]  # filtering out the test theta array.
        results = np.array(results)
        pd_dict = {
            f"perf_env{env_idx}": results[:, env_idx] for env_idx in range(args.num_envs)
        }
        pd_dict.update(
            {f"test_theta_env{env_idx}": combo[env_idx] for env_idx in range(args.num_envs)}
        )
        df = pd.DataFrame(data=pd_dict)
        dfs.append(df)

    df = pd.concat(dfs)
    try:
        save_runs("experiments", "utility-multi-env", df, args)
    except FileExistsError:
        pass
else:
    with open(os.path.join(exp_dir_load, "utility-multi-env", "results"), "rb") as f:
        df = pickle.load(f)

In [ ]:
df["test_theta_env0"] = df["test_theta_env0"].astype("category")
df["test_theta_env1"] = df["test_theta_env1"].astype("category")
pd.melt(df, id_vars="perf_env0", value_vars=("test_theta_env0", "test_theta_env1"))
df
df_ = df[["perf_env0", "test_theta_env0", "test_theta_env1"]]

df_["test_theta_env1"] = df_["test_theta_env1"].cat.rename_categories(
    {
        "theta_ao_hat": r"$\hat{\theta}^{AO}_2$",
        "theta_star_hat": r"$\hat{\theta}^*_2$",
        "theta_ols_hat": r"$\hat{\theta}_{OLS}$",
    }
)
df_ = df_.rename(columns={"test_theta_env1": r"$\theta_{2t}$"})
df_

df_test = df_.groupby([r"$\theta_{2t}$", "test_theta_env0"]).agg(["mean", "sem"])
df_test